# SSD Tutorial

This notebook provides a tutorial for running inference on a static image in order to detect objects.
The implementation of the [SingleShotDetectorLearner](https://github.com/opendr-eu/opendr/blob/master/docs/reference/object-detection-2d-ssd.md) is largely copied from its original
[implementation](https://github.com/dmlc/gluon-cv/tree/master/scripts/detection/ssd).
More information on modifications and license can be found
[here](https://github.com/opendr-eu/opendr/blob/master/src/opendr/perception/object_detection_2d/ssd/README.md).

First, we need to import the learner and initialize it:

In [ ]:
from opendr.perception.object_detection_2d import SingleShotDetectorLearner

learner = SingleShotDetectorLearner(device='cuda', backbone='vgg16_atrous')

Note that we can alter the device (e.g., 'cpu', 'cuda', etc.), on which the model runs, as well as the backbone
architecture from a variety of options (e.g. 'resnet50', 'resnet101', etc.).

After creating our model, we need to download pre-trained weights.

In [ ]:
learner.download(".")

and load our weights:

In [ ]:
learner.load("./ssd_default_person", verbose=True)

We will also download one sample image and load it, so we can use it in OpenDR for testing:

In [ ]:
learner.download(".", mode="images")

from opendr.engine.data import Image
img = Image.open("./people.jpg")

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
plt.imshow(cv2.cvtColor(img.opencv(), cv2.COLOR_BGR2RGB))

We are now ready to use our model!
The only thing that we have to do is to pass the image through the model.
Note that there are standard data types supported by OpenDR.
However, OpenDR also understands common data types (e.g,. OpenCV images) and automatically converts them into the most
appropriate format:

In [ ]:
detections = learner.infer(img)

We can plot the results using a utility function from the Object-Detection-2D module:

In [ ]:
from opendr.perception.object_detection_2d import draw_bounding_boxes

img_annotated = draw_bounding_boxes(img.opencv(), detections, class_names=learner.classes, show=False)


%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow(cv2.cvtColor(img_annotated, cv2.COLOR_BGR2RGB))